### AI model init 

In [1]:
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI
import os

from dotenv import load_dotenv
load_dotenv()

# if "GOOGLE_API_KEY" not in os.environ:
#     os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

# Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


# Local Ollama Model 
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage

# Replace this with your local Ollama model name
MODEL_NAME = "llama2"

# Initialize the Ollama chat model
# llm = ChatOllama(
#     model=MODEL_NAME,
#     base_url="http://localhost:11434",  # Default Ollama server URL
#     temperature=0,                   # Optional: Adjust sampling temperature
#     max_retries=3                      # Optional: Number of retries for failed requests
# )


# Example usage
# response = llm([HumanMessage(content="Explain the theory of relativity.")])
# print(response.content)


### init custom LLMGraphTransformer and neo4j instance 

In [2]:
from src.graph_builder.llm import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)



import os
from dotenv import load_dotenv
from src.graph_builder.lc_neo4j import Neo4jGraph
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "iambatman"
load_dotenv()
query_vector = llm_transformer._embed_text("")

graph = Neo4jGraph(refresh_schema=True,
                embedding_dim = len(query_vector)
)


Init LLMGraphTransformer


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INIT Neo4jGraph -5


### Read document and split and convert to GraphDocuments

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# 1) Read your full text (e.g. from PDF)
full_text = open("op.txt", encoding="utf-8").read()

# 2) Wrap in a single Document
original_doc = Document(page_content=full_text, metadata={"doc_id":"mydoc"})

# 3) Configure the splitter
splitter = RecursiveCharacterTextSplitter(
    # target chunk size in characters (not tokens)
    chunk_size=500,
    # how many characters of overlap between chunks
    chunk_overlap=100,
    # what to split on first, second, etc.
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
)

# 4) Produce a list of Documents, each with its own chunk
chunked_docs = splitter.split_documents([original_doc])

# Inspect
for i, doc in enumerate(chunked_docs[:3]):
    print(f"--- chunk {i} ({len(doc.page_content)} chars) ---")
    print(doc.page_content[:200].replace("\n"," "), "…\n")
print(len(chunked_docs))


graph_documents = llm_transformer.convert_to_graph_documents(chunked_docs)


--- chunk 0 (472 chars) ---
Neuro Prune: An Adaptive Approach for Efficient    Deep Neural Network Optimization on Edge    Devices    Dr. Karpagam G R1, Adhish Krishna S2, Mohana Kumar P3, Sanjay J4, Surya Narayanaa N T5    1Pro …

--- chunk 1 (444 chars) ---
sanjayjayakumar91@gmail.com , suryanarayanaant@gmail.com    Abstract—The optimization of deep neural networks for de-    ployment on edge devices is a significant research area due to    the demand fo …

--- chunk 2 (446 chars) ---
computational power, energy consumption, and latency. Pruning    is a method to reduce the model size, accelerate inference, and    save power. The objective of the paper is to propose the Neuro    Pr …

18
In -- convert_to_graph_documents


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Building final GraphDocument


### print pretty graph

In [4]:
def pretty_print_graph_documents(graph_documents):
    for idx, graph_doc in enumerate(graph_documents, start=1):
        print(f"\n### GraphDocument {idx}")
        print("**Nodes:**")
        for node_idx, node in enumerate(graph_doc.nodes, start=1):
            print(f"\n  {node_idx}. **Node ID:** {node.id}")
            print(f"     **Type:** {node.type}")
            print(f"     **Properties:**")
            for key, value in node.properties.items():
                if key == "source_texts":
                    print(f"       - **{key.capitalize()}:**")
                    for text in value:
                        print(f"         ```\n         {text}\n         ```")
                else:
                    print(f"       - **{key.capitalize()}:** {value}")

# Assuming `graph_documents` is the variable holding the output
pretty_print_graph_documents(graph_documents)
print(len(graph_documents))


### GraphDocument 1
**Nodes:**

  1. **Node ID:** b85404dfb4f434847a08e5ac0c17e749
     **Type:** Passage
     **Properties:**
       - **Text:** Neuro Prune: An Adaptive Approach for Efficient
   Deep Neural Network Optimization on Edge
   Devices
   Dr. Karpagam G R1, Adhish Krishna S2, Mohana Kumar P3, Sanjay J4, Surya Narayanaa N T5
   1Professor, Department of Computer Science and Engineering, PSG College of Technology
   grk.cse@psgtech.ac.in
   2,3,4,5BE-CSE (AI&ML), PSG College of Technology
   adhishthesak@gmail.com, mohanakumarp2828@gmail.com
   sanjayjayakumar91@gmail.com , suryanarayanaant@gmail.com
       - **Source_doc_id:** mydoc
       - **Source_page:** None
       - **Embedding:** [-0.012503384612500668, 0.06143878772854805, -0.00673447223380208, 0.025237757712602615, 0.014757942408323288, 0.033177461475133896, -0.017098233103752136, -0.005254596937447786, -0.03517308086156845, 0.026807203888893127, 0.027776500210165977, 0.001669572084210813, -0.035905372351408005, 0

### Add GraphDocuments to DB

In [5]:
graph.add_graph_documents(graph_documents,
                        include_source=True,             # Enable traceability
                        baseEntityLabel=True,
                        )

In [12]:
for doc in graph_documents:
    print(doc.relationships[0].properties['embedding'])   # Ensure embeddings are unique for each document
    print(doc.relationships[1].properties['embedding'])   # Ensure embeddings are unique for each document
    print(doc.relationships[2].properties['embedding'])   # Ensure embeddings are unique for each document


[-0.029524635523557663, 0.02821071818470955, -0.03181152418255806, 0.03932451456785202, -0.003279311815276742, -0.047103989869356155, 0.0496368370950222, 0.020443812012672424, -0.033717602491378784, 0.013925912790000439, -0.01716557703912258, 0.015231272205710411, -0.004539158660918474, 0.021805698052048683, -0.017529137432575226, 0.01560961827635765, 0.020639600232243538, 0.05385364219546318, 0.01659226045012474, -0.014793611131608486, -0.012477925978600979, -0.007249241229146719, 0.014675354585051537, 0.031223254278302193, -0.024090008810162544, 0.004843361210078001, 0.030760934576392174, 0.04833907261490822, 0.02213960327208042, -0.0086794663220644, 0.003023585770279169, 0.003872302360832691, 0.0006754331407137215, 0.010425611399114132, 2.4064411263680086e-06, 0.02582680433988571, 0.039550263434648514, 0.019349820911884308, -0.020786702632904053, 0.01306375116109848, 0.056766726076602936, -0.00025836200802586973, -0.0009719912777654827, -0.0014797807671129704, -0.011479047127068043,

In [28]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "iambatman"))

def get_all_node_embeddings(tx):
    query = "MATCH (n) WHERE n.embedding is NOT NULL RETURN id(n) AS id, n.embedding AS embedding, n.source_texts as source_texts"
    return list(tx.run(query))

def get_all_relationship_embeddings(tx):
    query = "MATCH ()-[r]->() WHERE r.embedding is NOT NULL RETURN id(r) AS id, r.embedding AS embedding,  r.source_texts as source_texts"
    return list(tx.run(query))

with driver.session() as session:
    node_results = session.read_transaction(get_all_node_embeddings)
    rel_results = session.read_transaction(get_all_relationship_embeddings)


C:\Users\SURYA\AppData\Local\Temp\ipykernel_21620\2219753826.py:15: DeprecationWarning: read_transaction has been renamed to execute_read
  node_results = session.read_transaction(get_all_node_embeddings)
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 1, column: 48, offset: 47} for query: 'MATCH (n) WHERE n.embedding is NOT NULL RETURN id(n) AS id, n.embedding AS embedding, n.source_texts as source_texts'
C:\Users\SURYA\AppData\Local\Temp\ipykernel_21620\2219753826.py:16: DeprecationWarning: read_transaction has been renamed to execute_read
  rel_results = session.read_transaction(get_all_relationship_embeddings)
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarni

In [34]:
len(node_results)

115

In [50]:
import numpy as np

# Suppose you have your query embedding as a numpy array
query_embedding = np.array([...])  # Your query embedding
query_embedding = llm_transformer._embed_text("accuracy of the model")
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# For nodes
node_similarities = []
for record in node_results:
    node_id = record["id"]
    embedding = record["embedding"]
    txt = record["source_texts"]
    sim = cosine_similarity(query_embedding, embedding)
    node_similarities.append((node_id, sim, txt))

# For relationships
rel_similarities = []
for record in rel_results:
    rel_id = record["id"]
    embedding = record["embedding"]
    sim = cosine_similarity(query_embedding, embedding)
    rel_similarities.append((rel_id, sim))

# Sort and get top-k
top_k = 10
top_nodes = sorted(node_similarities, key=lambda x: x[1], reverse=True)[:top_k]
top_rels = sorted(rel_similarities, key=lambda x: x[1], reverse=True)[:top_k]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [51]:
top_nodes

[(49,
  np.float64(0.3169185778458394),
  ['metric is derived to balance accuracy and sparsity using\n   a tunable parameter, λweight.\n   • Retrain the Model: The pruned model undergoes retrain-\n   ing to improve the loss in accuracy while maintaining low\n   computational requirements\n   Neuro Prune’s unique combination of reinforcement learn-\n   ing for adaptive decision-making and the Lottery Ticket Hy-\n   pothesis for identifying optimal subnetworks enables it to\n   achieve remarkable efficiency. This approach ensures substan-']),
 (50,
  np.float64(0.3169185778458394),
  ['metric is derived to balance accuracy and sparsity using\n   a tunable parameter, λweight.\n   • Retrain the Model: The pruned model undergoes retrain-\n   ing to improve the loss in accuracy while maintaining low\n   computational requirements\n   Neuro Prune’s unique combination of reinforcement learn-\n   ing for adaptive decision-making and the Lottery Ticket Hy-\n   pothesis for identifying optimal su